# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('task_time.npz') 

# Access the concatenated features from the .npz file
X_time = data['X_time']

# Close the loaded file
data.close()

#print the merged array
X_time

array([[[-1.58516986e-05, -1.83451157e-05,  4.28387813e-10, ...,
          4.28387813e-10,  1.15596322e-01,  5.15763781e+00],
        [-1.46279064e-06, -1.39965442e-06,  1.11612094e-10, ...,
          1.11612094e-10,  2.25178992e-01,  3.13333729e+00],
        [ 3.99905760e-06,  5.24598454e-06,  1.17426291e-10, ...,
          1.17426291e-10,  2.13541218e-01,  3.09660245e+00],
        ...,
        [ 1.79943197e-06,  3.93877440e-06,  1.18581890e-10, ...,
          1.18581890e-10,  2.25537207e-01,  2.81157912e+00],
        [ 4.63124429e-06,  5.90881791e-06,  1.34312102e-10, ...,
          1.34312102e-10,  2.13075454e-01,  3.19819047e+00],
        [-1.41503569e-06, -2.52987047e-06,  2.71039791e-10, ...,
          2.71039791e-10,  2.38988451e-01,  2.70227713e+00]],

       [[ 4.19056120e-07, -1.51430038e-06,  1.29960133e-10, ...,
          1.29960133e-10,  2.04071604e-01,  3.39227165e+00],
        [ 1.44862971e-06,  1.75205213e-06,  9.53881075e-11, ...,
          9.53881075e-11,  2.56205157e

In [3]:
data = np.load('task_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('task_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_time.shape, Y.shape, groups_array.shape)

(38049, 20, 13) (38049,) (38049,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [20]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [22]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_time, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_time[train_index], X_time[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandaradScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
134/134 [==============================] - 12s 38ms/step - loss: 0.2265 - accuracy: 0.9082 - val_loss: 0.1614 - val_accuracy: 0.9309
Epoch 2/200
134/134 [==============================] - 3s 24ms/step - loss: 0.1157 - accuracy: 0.9568 - val_loss: 0.0968 - val_accuracy: 0.9587
Epoch 3/200
134/134 [==============================] - 3s 23ms/step - loss: 0.0903 - accuracy: 0.9667 - val_loss: 0.0636 - val_accuracy: 0.9779
Epoch 4/200
134/134 [==============================] - 3s 23ms/step - loss: 0.0780 - accuracy: 0.9725 - val_loss: 0.0513 - val_accuracy: 0.9790
Epoch 5/200
134/134 [==============================] - 3s 24ms/step - loss: 0.0685 - accuracy: 0.9754 - val_loss: 0.0460 - val_accuracy: 0.9827
Epoch 6/200
134/134 [==============================] - 3s 23ms/step - loss: 0.0605 - accuracy: 0.9787 - val_loss: 0.0368 - val_accuracy: 0.9866
Epoch 7/200
134/134 [==============================] - 3s 24ms/step - loss: 0.0547 - accuracy: 0.9815 - val_loss: 0.0381 - val_a

In [23]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [24]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9989,0.9995,0.9986,0.9990,0.9990,0.9994,0.9986,0.9979,0.9979
1,0.9971,0.9962,0.9986,0.9974,0.9969,0.9953,0.9986,0.9942,0.9942
2,0.9955,0.9962,0.9957,0.9959,0.9955,0.9953,0.9957,0.9910,0.9910
3,0.9961,0.9967,0.9962,0.9964,0.9960,0.9959,0.9962,0.9920,0.9920
4,0.9955,0.9938,0.9981,0.9959,0.9952,0.9924,0.9981,0.9910,0.9910
5,0.9966,0.9952,0.9986,0.9969,0.9964,0.9942,0.9986,0.9931,0.9931
6,0.9971,0.9962,0.9986,0.9974,0.9969,0.9953,0.9986,0.9942,0.9942
7,0.9966,0.9957,0.9981,0.9969,0.9964,0.9947,0.9981,0.9931,0.9931
8,0.9955,0.9952,0.9967,0.9959,0.9954,0.9942,0.9967,0.9910,0.9910
9,0.9950,0.9938,0.9971,0.9955,0.9948,0.9924,0.9971,0.9899,0.9899


In [25]:
metrics_df.round(4).to_csv('TIME_TASK_CNN_LSTM.csv', index = False)

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
result = pd.read_csv("TIME_TASK_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9989,0.9995,0.9986,0.9990,0.9990,0.9994,0.9986,0.9979,0.9979
1,0.9971,0.9962,0.9986,0.9974,0.9969,0.9953,0.9986,0.9942,0.9942
2,0.9955,0.9962,0.9957,0.9959,0.9955,0.9953,0.9957,0.9910,0.9910
3,0.9961,0.9967,0.9962,0.9964,0.9960,0.9959,0.9962,0.9920,0.9920
4,0.9955,0.9938,0.9981,0.9959,0.9952,0.9924,0.9981,0.9910,0.9910
5,0.9966,0.9952,0.9986,0.9969,0.9964,0.9942,0.9986,0.9931,0.9931
6,0.9971,0.9962,0.9986,0.9974,0.9969,0.9953,0.9986,0.9942,0.9942
7,0.9966,0.9957,0.9981,0.9969,0.9964,0.9947,0.9981,0.9931,0.9931
8,0.9955,0.9952,0.9967,0.9959,0.9954,0.9942,0.9967,0.9910,0.9910
9,0.9950,0.9938,0.9971,0.9955,0.9948,0.9924,0.9971,0.9899,0.9899


In [28]:
(result.mean()*100).round(2)

Accuracy       99.64
Precision      99.58
Recall         99.76
F1             99.67
Sensitivity    99.63
Specificity    99.49
ROC_AUC        99.76
Kappa          99.27
MCC            99.27
dtype: float64

In [29]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])